In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Literal ,Annotated 
from langchain_core.messages import HumanMessage,BaseMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import operator
from langgraph.checkpoint.memory import MemorySaver

In [2]:
load_dotenv()
model= ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [3]:
from langgraph.graph.message import add_messages
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage],add_messages]

In [4]:
def chatnode(state:ChatState):
    #take user query from state
    messages = state['messages']

    # send to llm 
    response = model.invoke(messages)
    # response store to state
    return {"messages":[response]}

In [5]:
checkpointer = MemorySaver()
graph = StateGraph(ChatState)


#Add Node
graph.add_node("chatnode",chatnode)

# add edges

graph.add_edge(START,"chatnode")
graph.add_edge("chatnode",END)

chatbot=graph.compile(checkpointer=checkpointer)

In [6]:
initial_state = {

    'messages': [HumanMessage(content='what is the capital on india')]
}
# result=chatbot.invoke(initial_state)

In [7]:
# result['messages'][0].content
# result['messages'][-1].content

In [9]:
thread_id = '1'
while True:
    user_message = input('Type Here:')
    print('User:' ,user_message)
    if user_message.strip().lower() in ['exit' ,'quit' , 'bye']:
        break
    config = {'configurable':{"thread_id":thread_id}}
    response =chatbot.invoke({'messages':[HumanMessage(content=user_message)]},config=config)
    print('AI', response['messages'][-1].content)

User: hi my name is abhinav


d:\Coding\Self-Coding\langgraph_practice\myenv\Lib\site-packages\langchain_google_genai\chat_models.py:2007: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


AI Hi Abhinav! It's nice to meet you.

I'm an AI, so I don't have a personal name like a human.

How can I help you today?
User: what is my name


d:\Coding\Self-Coding\langgraph_practice\myenv\Lib\site-packages\langchain_google_genai\chat_models.py:2007: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


AI Your name is Abhinav.
User: add 100  into 450


d:\Coding\Self-Coding\langgraph_practice\myenv\Lib\site-packages\langchain_google_genai\chat_models.py:2007: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


AI 100 + 450 = **550**
User: now add 50


d:\Coding\Self-Coding\langgraph_practice\myenv\Lib\site-packages\langchain_google_genai\chat_models.py:2007: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


AI Okay, so we take the previous result (550) and add 50 to it:

550 + 50 = **600**
User: exit


In [10]:
chatbot.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='', additional_kwargs={}, response_metadata={}, id='8d65c1a2-f993-495f-af40-94b3e5da07d5'), HumanMessage(content='hi my name is abhinav', additional_kwargs={}, response_metadata={}, id='b1a472ee-7ab7-4dd5-ac0f-23934ca41da4'), AIMessage(content="Hi Abhinav! It's nice to meet you.\n\nI'm an AI, so I don't have a personal name like a human.\n\nHow can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--1cd5e352-4702-45b6-acaa-061cdf7dcf8d-0', usage_metadata={'input_tokens': 8, 'output_tokens': 776, 'total_tokens': 784, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 735}}), HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}, id='de00e9e6-e978-48d7-9d25-b001bb0f4267'), AIMessage(content='Your name is Abhinav.', a